<a href="https://colab.research.google.com/github/Sajithpemarathna/Inventory-business-case/blob/main/notebooks/inventory_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>